### **Definir el directorio de trabajo**
Define el directorio de trabajo como la carpeta `data` de la carpeta compartida `DS4A-Team12` de Drive.
Este directorio debe contener el archivo de datos original `Sociodemografico.zip` (5945450 registros, 140 MB) en el directorio `raw_data` y el diccionario de datos `dtypes_sociodemo.json` en el directorio `data_dictionaries`.


In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import joblib

### **Lectura del archivo**

In [ ]:
time0 = time.time()
# Abre el diccionario con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
# Diccionario con categorías válidas (ordenadas) para cada variable categórica (ordinal)
with open('datatypes_dictionaries/tom_cat.json', 'r') as file:
  tom_cat = json.load(file)
with open('datatypes_dictionaries/dtypes_sociodemo.json', 'r') as file:
  dtypes_soc = json.load(file)
# Lee el archivo
tom = pd.read_parquet('clean_data/tomas.parquet')
tom = tom.astype(dtypes_vars_toma)
tom.info()

soc = pd.read_parquet('clean_data/sociodemo_filtered.parquet')
soc = soc.astype(dtypes_soc)
soc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19275081 entries, 0 to 19275080
Data columns (total 27 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   IdToma                          UInt32        
 1   Registro                        UInt32        
 2   Vigencia                        category      
 3   Toma                            category      
 4   Servicio                        category      
 5   FechaValoracionNutricional      datetime64[ns]
 6   EdadMeses                       float32       
 7   FechaMedicionPerimetroBraquial  datetime64[ns]
 8   MedicionPerimetroBraquial       float32       
 9   Peso                            float32       
 10  Talla                           float32       
 11  ZScoreTallaEdad                 float32       
 12  ZScorePesoEdad                  float32       
 13  ZScorePesoTalla                 float32       
 14  ZScoreIMC                       float32       
 

In [ ]:
primera_toma = pd.read_parquet('auxiliary_data/beneficiario_primera_toma.parquet')
reincidencia = pd.read_parquet('clean_data/tagged_data')

In [ ]:
# Lista de variables categóricas
cols_cat = [col for col in tom.columns if col in tom_cat.keys()]
# Lista de variables categóricas ordinales
cols_ord = ['Vigencia', 'Toma', 
            'EstadoTallaEdad', 'EstadoPesoEdad', 'EstadoPesoTalla', 'EstadoIMC']
cols_fecha = [col for col in tom.columns if col[:5] == 'Fecha']
cols_medidas = ['MedicionPerimetroBraquial', 'Peso', 'Talla']
cols_zscore = [col for col in tom.columns if col[:6] == 'ZScore']
cols_estado = [col for col in tom.columns if col[:6] == 'Estado']
cols_zscore_peso = [col for col in tom.columns if col[:10] == 'ZScorePeso']
cols_estado_peso = [col for col in tom.columns if col[:10] == 'EstadoPeso']
for col in cols_cat:
  if str(tom[col].dtype) == 'category':
    print(f'\n ** {col} **')
    print(tom[col].value_counts(normalize=True, dropna=False))


 ** Vigencia **
2018    0.355224
2019    0.325344
2017    0.319432
Name: Vigencia, dtype: float64

 ** Toma **
3      0.253525
2      0.247413
4      0.246545
1      0.231485
1N     0.004039
2N     0.003694
3N     0.003081
4N     0.002768
5N     0.002265
6N     0.001591
7N     0.001264
8N     0.000946
9N     0.000710
10N    0.000539
11N    0.000092
12N    0.000045
Name: Toma, dtype: float64

 ** Servicio **
HCB TRADICIONAL- COMUNITARIO (T)                                                                  0.258447
DESARROLLO INFANTIL EN MEDIO FAMILIAR SIN ARRIENDO - FAMILIAR INTEGRAL                            0.157157
CDI SIN ARRIENDO -  INSTITUCIONAL INTEGRAL                                                        0.109048
DESARROLLO INFANTIL EN MEDIO FAMILIAR CON ARRIENDO - FAMILIAR INTEGRAL                            0.092390
HOGARES INFANTILES - INSTITUCIONAL INTEGRAL                                                       0.063899
HCB FAMI-FAMILIAR TRADICIONAL                        

In [ ]:
time0 = time.time()# Analizar las dos tomas anteriores
cols_imp = ['FechaValoracionNutricional', 'EdadMeses',
       'FechaMedicionPerimetroBraquial', 'MedicionPerimetroBraquial', 'Peso',
       'Talla', 'ZScoreTallaEdad', 'ZScorePesoEdad', 'ZScorePesoTalla',
       'ZScoreIMC', 'EstadoTallaEdad', 'EstadoPesoEdad', 'EstadoPesoTalla',
       'EstadoIMC', 'PresentaCarneVacunacion', 'ControlesCrecimDesarrollo']
prof = 2
cols_his = [col + '-' + str(prof - i) for i in range(prof + 1) for col in cols_imp] 
his_list = list()
len_melted = len(cols_prof)

ids_recuperacion = set(reincidencia.loc[reincidencia['recuperacion'], 'IdBeneficiario'])
#nuevo_ben = True
#presenta_desnutricion = False
for i in range(prof, len(tom)):
  id = tom.loc[i, 'IdBeneficiario']
  if (tom.loc[i - prof, 'IdBeneficiario'] == id) and (id in ids_recuperacion):
    fragmento = tom[i - prof: i + 1].reset_index(drop=True).\
      reset_index()
    melted = fragmento.melt(id_vars='index', value_vars=cols_imp)
    melted['varname'] = melted.\
      apply(lambda x: x['variable'] + '-' + str(prof - x['index']), axis='columns')
    melted['IdBeneficiario'] = [id] * len_melted
    pivot = melted.pivot(index='IdBeneficiario', columns='varname', values='value').\
      reset_index(drop=False)
    his_list.append(pivot)
    if i % 1e6 == 0:
      print(f'Toma {i}/{len(tom)}, {len(his)} fragmentos extraidos: ' +
            f'{time.time() - time0:.2f} s.')
his = pd.concat(his_list, axis='index')

NameError: ignored

In [ ]:
his_prof_2.to_parquet('historical_data/historia_prof_2')

In [ ]:
his_prof_2 = pd.read_parquet('historical_data/historia_prof_2')

In [ ]:
his = his_prof_2[cols_his + ['IdBeneficiario']]

In [ ]:
#Quitar las fechas, 'PresentaCarneVacunacion',  'ControlesCrecimDesarrollo'
his = his.drop(columns=[col for col in his.columns if col[:5] == 'Fecha'])
his = his.drop(columns=[col for col in his.columns if col[:8] == 'Presenta'])
his = his.drop(columns=[col for col in his.columns if col[:9] == 'Controles'])
# Quitar las características que no se quieren predecir
quit = [col for col in his.columns if col[-1] == '0' \
        and col not in ('EstadoPesoTalla-0', 'EdadMeses-0')]
his = his.drop(columns=quit)

In [ ]:
his['tiempo'] = his['EdadMeses-0'] - his['EdadMeses-1']
his[]

In [ ]:
for col in cols_estado:
  his[col] = his[col].astype('category').\
    cat.set_categories(tom_cat[col[:-2]], ordered=True)

In [ ]:
his['CodigoPesoTalla'] = his['EstadoPesoTalla-0'].cat.codes
his['Desnutricion'] = his['CodigoPesoTalla'].isin([0, 1])

In [ ]:
idB_desnutricion = his[his['Desnutricion']]['IdBeneficiario']
his = his[his['IdBeneficiario'].isin(idB_desnutricion)]

In [ ]:
# Cruza con sociodemográficos (se pierden la mitad de los datos)
join = soc.merge(his, how='inner', on='IdBeneficiario')
join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24711 entries, 0 to 24710
Data columns (total 97 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Cod_clase                    24711 non-null  category      
 1   Ind_grupo_sisben_4           24711 non-null  category      
 2   Ind_nivel_sisben_4           24711 non-null  category      
 3   Tip_vivienda                 24711 non-null  category      
 4   Ind_tiene_energia            24711 non-null  category      
 5   Ind_tiene_alcantarillado     24711 non-null  category      
 6   Ind_tiene_gas                24711 non-null  category      
 7   Ind_tiene_recoleccion        24711 non-null  category      
 8   Ind_tiene_acueducto          24711 non-null  category      
 9   n_hogares_vivienda           24711 non-null  UInt8         
 10  Tip_ocupa_vivienda           24711 non-null  category      
 11  Ind_tiene_cocina             24711 non-nu

In [ ]:
# Descarta tomas muy seguidas o fechas absurdas
his = his[his['tiempo'] > 0]
his_tiempo = list()
for i in range(4):
  his_tiempo.append(his[his['tiempo'] == i + 1].drop(columns=['tiempo']))

join = join[join['tiempo'] > 0]
join_tiempo = list()
for i in range(4):
  join_tiempo.append(join[join['tiempo'] == i + 1].drop(columns=['tiempo']))

his['tiempo'].value_counts()[:10]

3.0     13739
2.0      4201
4.0      3088
1.0      2175
5.0       348
6.0       234
7.0       125
9.0        94
10.0       67
Name: tiempo, dtype: int64

In [ ]:
# Realiza conjuntos de entrenamiento y prueba para este conjunto
IdB_train = joblib.load('auxiliary_data/IdBeneficiario_train.pickle')
IdB_test = joblib.load('auxiliary_data/IdBeneficiario_test.pickle')


his_train = his[his['IdBeneficiario'].isin(IdB_train)]
his_test = his[his['IdBeneficiario'].isin(IdB_test)]
print(len(his_train), len(his_test))

join_train = join[join['IdBeneficiario'].isin(IdB_train)]
join_test = join[join['IdBeneficiario'].isin(IdB_test)]
print(len(join_train), len(join_test))

19729 5002
19714 4997


In [ ]:
join.to_parquet('historical_data/sociodemo_historia_2.parquet')
join_train.to_parquet('historical_data/sociodemo_historia_2_train.parquet')
join_test.to_parquet('historical_data/sociodemo_historia_2_test.parquet')

his.to_parquet('historical_data/historia_2.parquet')
his_train.to_parquet('historical_data/historia_2_train.parquet')
his_test.to_parquet('historical_data/historia_2_test.parquet')